In [3]:
import numpy
import pandas as pd
import csv
import matplotlib.pyplot as plt
import os

In [18]:
folder_bench='results-'
folder_cass='results-cassandra'
experiment_date='2019-09-26T174934'
files = []

In [19]:
folder=folder_bench+experiment_date
with os.scandir(folder) as it:
    for file in it:
        if file.name.endswith('.csv') & file.is_file():
            files.append(folder+'/'+file.name)

## We need some transformations

1. Calculate Span Duration from FinishTime - StartTime

In [27]:
files
data = pd.concat([pd.read_csv(f) for f in files])
traces = pd.read_csv(folder_cass+'/'+'traces.csv')

In [36]:
dft = pd.DataFrame(traces)
df = pd.DataFrame(data)
df['SpanDuration'] = df.FinishTime - df.StartTime
df.head(10)

,Service,TraceNumber,SpanNumber,StartTime,FinishTime,Sampled,SpanDuration
0,svc02,0,-1,1569512974961028894,1569512974981172140,False,20143246
1,svc02,1,-1,1569512975061055150,1569512975081219275,False,20164125
2,svc02,2,-1,1569512975161338263,1569512975181510639,False,20172376
3,svc02,3,-1,1569512975261222211,1569512975281365677,False,20143466
4,svc02,4,-1,1569512975361122528,1569512975381242354,False,20119826
5,svc02,5,-1,1569512975460976566,1569512975481133461,False,20156895
6,svc02,6,-1,1569512975561016572,1569512975581197477,False,20180905
7,svc02,7,-1,1569512975661348596,1569512975681979627,False,20631031
8,svc02,8,-1,1569512975761485650,1569512975781600308,False,20114658
9,svc02,9,-1,1569512975861189670,1569512975881380836,False,20191166


In [37]:
df_transformed = df_sorted.copy()
df_transformed.StartTime = df_sorted.StartTime / 1000.0
df_transformed.FinishTime = df_sorted.FinishTime / 1000.0
df_transformed = df_transformed.astype({'StartTime': 'int64', 'FinishTime': 'int64'})
df_transformed = df_transformed.loc[df_transformed['Sampled']]
df_transformed = df_transformed.sort_values(by = 'StartTime')
dft_sorted = dft.sort_values(by = 'start_time')
df_transformed.head(10)

,Service,TraceNumber,SpanNumber,StartTime,FinishTime,Sampled,SpanDuration
31,svc01,15,-1,1569512976440365,1569512976501469,True,61103520
30,svc01,15,0,1569512976440393,1569512976481318,True,40925016
15,svc02,15,-1,1569512976460891,1569512976481050,True,20159155
33,svc01,16,-1,1569512976540394,1569512976601721,True,61326859
32,svc01,16,0,1569512976540421,1569512976581543,True,41121503
16,svc02,16,-1,1569512976560945,1569512976581260,True,20314734
217,svc01,108,-1,1569512985741163,1569512985802845,True,61682165
216,svc01,108,0,1569512985741194,1569512985782103,True,40908975
108,svc02,108,-1,1569512985761597,1569512985781853,True,20256255
233,svc01,116,-1,1569512986540314,1569512986601069,True,60755254


In [35]:
dft_sorted.head(10)

,trace_id,span_id,span_hash,duration,flags,logs,operation_name,parent_id,process,refs,start_time,tags,write_time
66,0x00000000000000000564c90bad9e8f8c,388656519855378316,3200405408741421690,60921,1,"[{ts: 1569512599416836, fields: [{key: 'event'...",svc01-parent,0,"{service_name: 'svc01', tags: [{key: 'jaeger.v...",NaN,1569512599416815,"[{key: 'sampler.type', value_type: 'string', v...",1569512600323793
68,0x00000000000000000564c90bad9e8f8c,9153216720797684114,-7988554849494823841,40786,1,NaN,svc01-call-0,0,"{service_name: 'svc01', tags: [{key: 'jaeger.v...","[{ref_type: 'child-of', trace_id: 0x0000000000...",1569512599416844,"[{key: 'internal.span.format', value_type: 'st...",1569512600323766
67,0x00000000000000000564c90bad9e8f8c,1065153142657624982,6791136895179182198,20086,1,NaN,svc02-parent,0,"{service_name: 'svc02', tags: [{key: 'jaeger.v...","[{ref_type: 'child-of', trace_id: 0x0000000000...",1569512599437326,"[{key: 'j6Ia3uoFqv1yrQMXLH40', value_type: 'st...",1569512600323642
81,0x0000000000000000017f5ca7c839ad2b,107906791768698155,8967717604219671240,61100,1,"[{ts: 1569512599616854, fields: [{key: 'event'...",svc01-parent,0,"{service_name: 'svc01', tags: [{key: 'jaeger.v...",NaN,1569512599616828,"[{key: 'sampler.type', value_type: 'string', v...",1569512600323869
83,0x0000000000000000017f5ca7c839ad2b,8612864633337554759,-6951797603499308089,40925,1,NaN,svc01-call-0,0,"{service_name: 'svc01', tags: [{key: 'jaeger.v...","[{ref_type: 'child-of', trace_id: 0x0000000000...",1569512599616864,"[{key: 'internal.span.format', value_type: 'st...",1569512600323694
82,0x0000000000000000017f5ca7c839ad2b,3005815224900208354,6312906832042416801,20134,1,NaN,svc02-parent,0,"{service_name: 'svc02', tags: [{key: 'jaeger.v...","[{ref_type: 'child-of', trace_id: 0x0000000000...",1569512599637406,"[{key: 'j6Ia3uoFqv1yrQMXLH40', value_type: 'st...",1569512600323568
60,0x000000000000000007a88841764c4ab5,551840769091848885,-7157995111786673923,61162,1,"[{ts: 1569512600616837, fields: [{key: 'event'...",svc01-parent,0,"{service_name: 'svc01', tags: [{key: 'jaeger.v...",NaN,1569512600616816,"[{key: 'sampler.type', value_type: 'string', v...",1569512601317184
61,0x000000000000000007a88841764c4ab5,3590189633508744849,-461095317084713604,40994,1,NaN,svc01-call-0,0,"{service_name: 'svc01', tags: [{key: 'jaeger.v...","[{ref_type: 'child-of', trace_id: 0x0000000000...",1569512600616847,"[{key: 'internal.span.format', value_type: 'st...",1569512601317149
62,0x000000000000000007a88841764c4ab5,8140743362983686138,-4484787145657179047,20099,1,NaN,svc02-parent,0,"{service_name: 'svc02', tags: [{key: 'jaeger.v...","[{ref_type: 'child-of', trace_id: 0x0000000000...",1569512600637352,"[{key: 'j6Ia3uoFqv1yrQMXLH40', value_type: 'st...",1569512601316584
162,0x000000000000000000e967697b7e6ee3,65697372314693347,7697307403657342859,61269,1,"[{ts: 1569512601516896, fields: [{key: 'event'...",svc01-parent,0,"{service_name: 'svc01', tags: [{key: 'jaeger.v...",NaN,1569512601516858,"[{key: 'sampler.type', value_type: 'string', v...",1569512602316130
